# PHÂN TÍCH CHUỖI CUNG ỨNG VÀ THỜI TIẾT

Notebook này thể hiện các phân tích và công việc đã thực hiện trong dự án Supply Chain Analytics Dashboard.

---


## 1. IMPORT THƯ VIỆN VÀ CẤU HÌNH


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✓ Đã import các thư viện cần thiết")


## 2. ĐỌC DỮ LIỆU


In [ ]:
# Đường dẫn file
import os
import sys

# Thêm thư mục app vào path
sys.path.insert(0, os.path.join('..', 'app'))

from services.data_loader import load_supply_chain_data, load_weather_data

print("Đang đọc dữ liệu...")

# Đọc dữ liệu
supply_df = load_supply_chain_data()
weather_df = load_weather_data()

print(f"\n✓ Đã đọc Supply Chain data: {len(supply_df):,} bản ghi")
print(f"✓ Đã đọc Weather data: {len(weather_df):,} bản ghi")


### 2.1. Tổng quan dữ liệu Supply Chain


In [ ]:
print("=== TỔNG QUAN SUPPLY CHAIN DATASET ===")
print(f"Số dòng: {len(supply_df):,}")
print(f"Số cột: {len(supply_df.columns)}")
print(f"\nCác cột quan trọng:")
important_cols = ['Order Id', 'Sales', 'Benefit per order', 'Delivery Status', 
                  'Late_delivery_risk', 'Order Country', 'Category Name', 
                  'order date (DateOrders)']
for col in important_cols:
    if col in supply_df.columns:
        print(f"  - {col}")

print("\n=== 5 DÒNG ĐẦU TIÊN ===")
supply_df.head()


## 3. TÍNH TOÁN KPI


In [ ]:
from services.analytics import calculate_supply_chain_kpis

# Tính KPI
kpis = calculate_supply_chain_kpis(supply_df)

print("=== CÁC KPI CHÍNH ===")
print(f"Tổng doanh thu: ${kpis.get('total_sales', 0):,.2f}")
print(f"Tổng lợi nhuận: ${kpis.get('total_benefit', 0):,.2f}")
print(f"Tổng số đơn hàng: {kpis.get('total_orders', 0):,}")
print(f"Tỉ lệ giao trễ: {kpis.get('late_delivery_rate', 0):.2f}%")
print(f"Tỉ lệ giao đúng hạn: {kpis.get('on_time_delivery_rate', 0):.2f}%")
print(f"Số ngày giao hàng trung bình: {kpis.get('avg_shipping_days', 0):.2f} ngày")


## 4. TOP PRODUCTS VÀ COUNTRIES


In [ ]:
from services.analytics import get_top_products, get_top_countries

# Top products
top_products = get_top_products(supply_df, top_n=10, by='Sales')
top_countries = get_top_countries(supply_df, top_n=10, by='Sales')

print("=== TOP 10 SẢN PHẨM THEO DOANH THU ===")
for i, product in enumerate(top_products, 1):
    print(f"{i}. {product['category']}: ${product['sales']:,.2f}")

print("\n=== TOP 10 QUỐC GIA THEO DOANH THU ===")
for i, country in enumerate(top_countries, 1):
    print(f"{i}. {country['country']}: ${country['sales']:,.2f} ({country['orders']:,} đơn)")


## 5. PHÂN TÍCH TIME SERIES


In [ ]:
from services.analytics import get_time_series_data

# Lấy dữ liệu time series theo tháng
time_series = get_time_series_data(supply_df, freq='M')

print("=== TIME SERIES DATA ===")
print(f"Số tháng có dữ liệu: {len(time_series.get('sales', {}))}")

# Chuyển đổi sang DataFrame để dễ visualize
if time_series.get('sales'):
    ts_df = pd.DataFrame({
        'Date': pd.to_datetime(list(time_series['sales'].keys())),
        'Sales': list(time_series['sales'].values()),
        'Orders': list(time_series.get('orders_count', {}).values()) if time_series.get('orders_count') else [0] * len(time_series['sales']),
        'Late Rate': list(time_series.get('late_delivery_rate', {}).values()) if time_series.get('late_delivery_rate') else [0] * len(time_series['sales'])
    })
    ts_df = ts_df.sort_values('Date')
    
    print("\n5 tháng đầu tiên:")
    print(ts_df.head())
    
    print("\n5 tháng cuối cùng:")
    print(ts_df.tail())


## 6. PHÂN TÍCH TƯƠNG QUAN THỜI TIẾT


In [ ]:
from services.analytics import analyze_weather_delivery_correlation

# Phân tích tương quan
correlation = analyze_weather_delivery_correlation(supply_df, weather_df)

print("=== KẾT QUẢ PHÂN TÍCH TƯƠNG QUAN ===")
if 'error' not in correlation:
    print(f"Số bản ghi đã merge: {correlation.get('merged_records', 0):,}")
    print(f"Tỉ lệ merge: {correlation.get('merge_rate', 0):.2f}%")
    print(f"\nHệ số tương quan:")
    if correlation.get('temperature_vs_late_delivery'):
        print(f"  - Nhiệt độ vs Giao trễ: {correlation.get('temperature_vs_late_delivery'):.4f}")
    if correlation.get('precipitation_vs_late_delivery'):
        print(f"  - Lượng mưa vs Giao trễ: {correlation.get('precipitation_vs_late_delivery'):.4f}")
    if correlation.get('wind_speed_vs_late_delivery'):
        print(f"  - Tốc độ gió vs Giao trễ: {correlation.get('wind_speed_vs_late_delivery'):.4f}")
else:
    print(f"Lỗi: {correlation.get('error')}")
    if 'suggestion' in correlation:
        print(f"Gợi ý: {correlation.get('suggestion')}")


## 7. KẾT LUẬN VÀ INSIGHTS

### Tóm tắt Insights

1. **KPI Tổng quan:**
   - Tổng doanh thu và lợi nhuận
   - Tỉ lệ giao hàng trễ
   - Số đơn hàng

2. **Top Performers:**
   - Top sản phẩm và quốc gia theo doanh thu
   - Phân tích xu hướng

3. **Time Series:**
   - Xu hướng doanh thu, số đơn hàng, tỉ lệ giao trễ theo thời gian
   - Phát hiện patterns và seasonality

4. **Weather Impact:**
   - Tương quan giữa thời tiết và giao hàng
   - Phân tích tác động của nhiệt độ, lượng mưa, tốc độ gió

### Đề xuất cải tiến

1. **Chuẩn hóa dữ liệu:** Tên quốc gia, format ngày tháng, missing values
2. **Cải thiện join:** Tạo bảng mapping city/country → lat/lon
3. **Star Schema:** Tách fact và dimension tables
4. **Feature Store:** Chuẩn bị cho ML models

Xem chi tiết trong file `docs/data_improvement_plan.md`

---

**Notebook này thể hiện các phân tích và công việc đã thực hiện trong dự án Supply Chain Analytics Dashboard.**

**Để chạy dashboard web, xem hướng dẫn trong `docs/README_dashboard.md`**
